In [ ]:
#!pip install fuzzywuzzy
#!pip install python-Levenshtein
#nltk.download('wordnet')

In [160]:
import pandas as pd
import numpy as np
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
import re
lemmatizer = WordNetLemmatizer() 

In [476]:
with open("Toyota_Highlander_1.txt") as file:
    data11=file.read()
with open("Toyota_Highlander_2.txt") as file:
    data12=file.read()
with open("Toyota_Highlander_3.txt") as file:
    data13=file.read()
data = pd.read_csv("nissan_edmunds.csv")
data1 = pd.read_excel("Highlander.xlsx",header=None)


In [39]:
data1

,0
0,"If you're shopping within the Toyota family, t..."
1,The 2019 Toyota Highlander ranks highly in the...
2,"If you're shopping within the Toyota family, t..."


In [65]:
features = pd.DataFrame()
with open("FeaturesCar.txt","r") as file:
    for line in file:
        features=features.append([line.split("\n")[0]],ignore_index=True)
file.close()

In [67]:
features.to_csv("features.csv",header=None,index=None)

In [522]:
#nltk.pos_tag(features[0].tolist())

[('Apple', 'NNP'), ('carplay', 'NN')]

In [440]:
stop_words = set(stopwords.words('english'))
stop_words.add(",")
stop_words.add(".")
stop_words.add("!")
stop_words.add("(")
stop_words.add(")")

In [441]:
def findadv(sentance,feature,match,prox_noun=4,prox_adj=4):
    tagged_words=nltk.pos_tag(sentance)
    adj=[]
    quality=""
    index=0
    for i in range(len(tagged_words)):
        if fuzz.ratio(tagged_words[i][0].lower(),feature.lower())>match:
            index = i
            quality= tagged_words[i][0]
            j=0
            #Here we are finding nouns in the proximinty of 4 words if they are noun we are adding it to feature name
            # -4 words
            while i-j>0 and j<prox_noun:
                j=j+1
                if re.search(r'NN.*',tagged_words[i-j][1]):
                    quality=tagged_words[i-j][0]+" "+quality
                else:
                    break
            #Here we are finding Adjectives which may be the value of feature in proximity of 4 (default)
            # -4 words
            if (i-j)>0:
                ad=0
                while i-j-ad>0 and ad<prox_adj:

                    if re.search(r'JJ.*',tagged_words[i-j-ad][1]):
                        adj.append(tagged_words[i-j-ad][0])
                    ad=ad+1
             #Here we are finding nouns in the proximinty of 4 words if they are noun we are adding it to feature name
            # +4 words  
            j=0
            while i+j<len(tagged_words)-1 and j<prox_noun:
                j=j+1
                if re.search(r'NN.*',tagged_words[i+j][1]):
                    quality=quality+" "+tagged_words[i+j][0]
                else:
                    break
                    
            if (i+j)<len(tagged_words):
                ad=0
                while i+j+ad<len(tagged_words)-1 and ad<prox_adj:

                    if re.search(r'JJ.*',tagged_words[i+j+ad][1]):
                        adj.append(tagged_words[i+j+ad][0])
                    ad=ad+1
            break
    return [quality,adj]

In [442]:
# sentances = nltk.sent_tokenize(data1[0][0])
# for feature in features[0].tolist():
#     for sentance in sentances:
#         word_tokens= nltk.word_tokenize(sentance)                                   #      tokenizer.tokenize(data1[0][0])
#         filtered_sentence = [w for w in word_tokens if not w in stop_words]


#         for word in filtered_sentence:

#             if fuzz.ratio(word.lower(),feature.lower())>80:
#                 print(findadv(filtered_sentence, feature))

In [445]:
def findcarfeatures(features,document,prox_noun=4,prox_adj=4,match=80):
    
    """Takes list of features and document in string formate"""
    result = []
    sentances = nltk.sent_tokenize(document)
    for feature in features:
        for sentance in sentances:
            word_tokens= nltk.word_tokenize(sentance)                                   #      tokenizer.tokenize(data1[0][0])
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            for word in filtered_sentence:
                if fuzz.ratio(word.lower(),feature.lower())>match:
                    result.append(findadv(word_tokens, feature,match))
    return result
    

In [479]:
findcarfeatures(features[0].tolist(),data1[0][0],prox_noun=4,prox_adj=4,match=80)

[['steering wheel', ['leather-wrapped']],
 ['alloy wheels', ['18-inch', 'three-zone', 'automatic']],
 ['steering wheel', ['leather-wrapped']],
 ['wheels', ['20-inch', 'Limited', 'handsfree']],
 ['LED headlights', ['adaptive']],
 ['headlights', ['automatic']],
 ['LED headlights', []],
 ['headlights', ['self-leveling', 'adaptive', 'automatic']],
 ['touchscreen', ['8-inch']],
 ['touchscreen', ['12.3-inch']],
 ['safety section', ['full-speed']]]

In [473]:
findcarfeatures(features[0].tolist(),data11,prox_noun=4,prox_adj=4,match=80)

[['V6 engine', ['previous', 'standard']],
 ['dash', ['high', 'easy']],
 ['infotainment features', ['different']],
 ['touchscreen', ['12.3-inch', 'jumbo', 'available']],
 ['touchscreen', ['eight-inch']],
 ['safety equipment', ['generous', 'well-equipped']]]

In [477]:
findcarfeatures(features[0].tolist(),data12,prox_noun=4,prox_adj=4,match=80)

[['V6 engine', ['optional', 'efficient', 'powerful']],
 ['V6 engine', ['optional']],
 ['V6 engine', ['powerful']],
 ['V6 engine', ['all-wheel']],
 ['V6 engine', ['come']],
 ['suspension system', ['sport-tuned', 'black', 'leather-trimmed']],
 ['alloy wheels', ['19-inch', 'sport-tuned']],
 ['straightforward infotainment system', ['standard']],
 ['safety features', ['standard', 'impressive']],
 ['safety features', ['standard', 'generous']],
 ['Highway Safety', []]]

In [478]:
findcarfeatures(features[0].tolist(),data13,prox_noun=4,prox_adj=4,match=80)

[['steering wheel', ['leather-wrapped']],
 ['alloy wheels', ['18-inch', 'three-zone', 'automatic']],
 ['steering wheel', ['leather-wrapped']],
 ['wheels', ['20-inch', 'Limited', 'handsfree']],
 ['LED headlights', ['adaptive']],
 ['headlights', ['automatic']],
 ['LED headlights', []],
 ['headlights', ['self-leveling', 'adaptive', 'automatic']],
 ['touchscreen', ['8-inch']],
 ['touchscreen', ['12.3-inch']],
 ['safety section', ['full-speed']]]

In [434]:
nltk.sent_tokenize(data1[0][0])

["If you're shopping within the Toyota family, the Highlander offers appreciably more space than the RAV4, both in terms of cargo capacity and its extra row of seats.",
 "It also has a deeper, more accessible space than what's in the 4Runner.",
 'That said, the Highlander is one of the smallest three-row crossovers available.',
 "Apart from the Kia Sorento and maybe the Mazda CX-9, you're going to find more cargo capacity and passenger space in the Highlander's competitors.",
 "That's especially true in the third row.",
 "The second row slides a bit more to grant extra legroom now, but the third row remains awfully close to the floor, and it won't be long before your growing kids will feel cramped and claustrophobic in the way-back.",
 'Full-size teens and adults will be flat-out grumpy.',
 "That said, the Highlander's smaller size might be just right for many buyers who appreciate its more manageable dimensions when parking or maneuvering in tight spots.",
 "Plus, if you only need tha

In [480]:
data1[0][0]

"If you're shopping within the Toyota family, the Highlander offers appreciably more space than the RAV4, both in terms of cargo capacity and its extra row of seats. It also has a deeper, more accessible space than what's in the 4Runner.\nThat said, the Highlander is one of the smallest three-row crossovers available. Apart from the Kia Sorento and maybe the Mazda CX-9, you're going to find more cargo capacity and passenger space in the Highlander's competitors. That's especially true in the third row. The second row slides a bit more to grant extra legroom now, but the third row remains awfully close to the floor, and it won't be long before your growing kids will feel cramped and claustrophobic in the way-back. Full-size teens and adults will be flat-out grumpy.\nThat said, the Highlander's smaller size might be just right for many buyers who appreciate its more manageable dimensions when parking or maneuvering in tight spots. Plus, if you only need that third row for occasional use 

In [528]:
def findcarfeatures2(features,document,prox_noun=4,prox_adj=4,match=80):
    
    """Takes list of features and document in string formate"""
    result = []
    featuresIdentfied=[]
    sentances = nltk.sent_tokenize(document)
    
    for feature in features:
        lenfeature = len(feature.split(" "))
        for sentance in sentances:
            word_tokens= nltk.word_tokenize(sentance)                                   #      tokenizer.tokenize(data1[0][0])
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            for i in range(len(word_tokens)-lenfeature+1):
                wordtocomapre=""
                for j in range(i,i+lenfeature):
                    if re.search(r'[,()]',word_tokens[j]):
                        break
                    wordtocomapre=wordtocomapre+" "+word_tokens[j].lower()
                wordtocomapre=wordtocomapre.strip()
                if not wordtocomapre=="":
                    if fuzz.ratio(wordtocomapre,feature.lower())>match:
                        result.append([wordtocomapre,feature])
                        #featuresIdentfied.append(findadv(word_tokens, feature, match))
    return result,featuresIdentfied

In [531]:
findcarfeatures2(features[0].tolist(),data1[0][2],prox_noun=4,prox_adj=4)

([['steering', 'STEERING'],
  ['cruise control', 'CRUISE CONTROL'],
  ['blind-spot warning', 'BLIND SPOT WARNING'],
  ['rearview mirror', 'REARVIEW MONITOR'],
  ['rearview mirror', 'REARVIEW MONITOR'],
  ['wheels', 'WHEELS'],
  ['wheel', 'WHEELS'],
  ['wheels', 'WHEELS'],
  ['headlights', 'HEADLIGHTS'],
  ['headlights', 'HEADLIGHTS'],
  ['headlights', 'HEADLIGHTS'],
  ['headlights', 'HEADLIGHTS'],
  ['automatic headlights', 'AUTOMATIC ON/OFF HEADLIGHTS'],
  ['automatic climate control', 'AUTOMATIC TEMPERATURE CONTROL'],
  ['push-button start', 'Push-Button Start'],
  ['heated front seats', 'HEATED FRONT SEATS'],
  ['ventilated front seats', 'HEATED FRONT SEATS'],
  ['a leather-wrapped steering', 'LEATHER-WRAPPED STEERING WHEEL'],
  ['leather-wrapped steering wheel', 'LEATHER-WRAPPED STEERING WHEEL'],
  ['touchscreen', 'TOUCH-SCREEN'],
  ['touchscreen', 'TOUCH-SCREEN'],
  ['safety', 'SAFETY'],
  ['cargo capacity', 'CARGO CAPACITY'],
  ['cargo capacity', 'CARGO CAPACITY']],
 [])

In [468]:
findcarfeatures2(features[0].tolist(),data1[0][1],prox_noun=4,prox_adj=4)

({'automatic climate control',
  'engine',
  'infotainment',
  'safety',
  'suspension',
  'usb ports',
  'wheels'},
 [])

In [462]:
data1[0][0]

"If you're shopping within the Toyota family, the Highlander offers appreciably more space than the RAV4, both in terms of cargo capacity and its extra row of seats. It also has a deeper, more accessible space than what's in the 4Runner.\nThat said, the Highlander is one of the smallest three-row crossovers available. Apart from the Kia Sorento and maybe the Mazda CX-9, you're going to find more cargo capacity and passenger space in the Highlander's competitors. That's especially true in the third row. The second row slides a bit more to grant extra legroom now, but the third row remains awfully close to the floor, and it won't be long before your growing kids will feel cramped and claustrophobic in the way-back. Full-size teens and adults will be flat-out grumpy.\nThat said, the Highlander's smaller size might be just right for many buyers who appreciate its more manageable dimensions when parking or maneuvering in tight spots. Plus, if you only need that third row for occasional use 

In [519]:

from spacy.matcher import PhraseMatcher
import spacy
nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)
matcher.add("SCREEN", None, nlp(u"Touchscreen"))
doc = nlp(u"Touchscreens are most comman these days")
matches = matcher(doc)
matches

[]

In [ ]:
fuzz.partial_ratio("drivetrain","drive train")

In [13]:
process.extract("wiper",data.loc[0,"corpus"].split(" "))

[('wipers', 91), ('wipers', 91), ('were', 67), ('We', 57), ('we', 57)]

In [14]:
for row in data.corpus:
    response=process.extract("wipers",row.split(" "),scorer=fuzz.QRatio)
    for i,j in response:
        if j>70:
            print(i)
            break
        

wipers
wipers
wiper
wipers
wipers
wiped
wipes


In [15]:
file = open("FeaturesCar.txt","r")
reader = file.readlines()

In [17]:
data.loc[0,"corpus"]
    

'The wipers were almost useless. They left such bad streaks that driving was dangerous  especially in the dark. We had to purchase new wipers so we could see.'